In [18]:
using Distributions, Plots, Random
include("../src/JS_SAA_main.jl")
pyplot()

Plots.PyPlotBackend()

In [19]:
#discrete newsvendors supported 1:d
K = 10000
s = .91
d = 75
N = 10

Random.seed!(8675309)
ps = rand(Dirichlet(ones(d)), floor(Int, K/2))
ps = [ps rand(Dirichlet(.025 * ones(d)), K - floor(Int, K/2)) ]

# #gen an "interesting" distribution of ps still centered at 1/d
# ps = rand(Dirichlet(5 * ones(d)), floor(Int, K/2))
# qs = rand(Dirichlet(5 * ones(d)), K-floor(Int, K/2))

# qanchor = vcat(.9 * [.2, .2, .2, .2, .2], .1 * ones(d-5)/(d-5))
# qs = rand(Dirichlet(qanchor),  K - floor(Int, K/2)) 

# ps = [ps qs]

alpha_grid = range(0, stop=3N, length=100)
lams = ones(K)


Nhats = rand(Poisson(N), K)
mhats = JS.sim_path(ps, Nhats);

p0 = JS.get_GM_anchor(mhats)
supps =  repeat(collect(1:d), outer=(1, K))
cs, xs = JS.genNewsvendorsDiffSupp(supps, s, K);

In [31]:
alpha_grid = range(0, stop=30, length=100)
# @time alphaOR, jstar, outOR = JS.oracle_alpha(xs, cs, mhats, ps, ones(K), p0, alpha_grid)
# @time alphaLOO, jstar, outLOO = JS.loo_alpha(xs, cs, mhats, p0, alpha_grid)
# @time outSAA = map(a->JS.zbar(xs, cs,mhats, mhats ./ Nhats', ones(K),  (p0, a)), alpha_grid);
@time alphaCV, jstar, outCV = JS.cv_alpha(xs, cs, mhats, p0, alpha_grid, 5)
# @time full_info = JS.zstar(xs, cs, ps, ones(K));

 59.005494 seconds (1.24 G allocations: 70.959 GiB, 25.10% gc time)


In [ ]:
# println("SAA:\t", outOR[1], "\t", outOR[1] / full_info - 1)
# println("LOO:\t", outOR[argmin(outLOO)], "\t", outOR[argmin(outLOO)]/full_info - 1)
# println("OR:\t", minimum(outOR), "\t", minimum(outOR)/full_info - 1)
plot(alpha_grid, [outOR outLOO/N outCV/N], label=["Oracle" "LOO" "CV"], marker=:circ)


In [ ]:
### Sanity Checks.  
#  ###Check that xs(alpha) actually minimizes p(\alpha)^T c(alpha)
#  # spot check zLOO(alpha) and ZOR(alpha) against other implementation
#  # do a run for large K.
#  #check that oracle value minimized at bayesian value = 2
#  #do a curve by curve spot check
k0 = 15
alpha0 = 0
palpha_k = JS.shrink(mhats[:, k0]/Nhats[k0], p0, alpha0, Nhats[k0])
xalpha_k = xs[k0](p0, alpha0, mhats[:, k0])

println("palpha_k:\t", palpha_k)
println("x_alphak:\t", xalpha_k)

dot(palpha_k, map(c_ik -> c_ik(9), cs[:, k0])), dot(palpha_k, map(c_ik -> c_ik(10), cs[:, k0]))


# alpha = 2
# palphas = similar(ps)
# for k = 1:K
#     palphas[:, k] = JS.shrink(mhats[:, k]/Nhats[k], p0, alpha, Nhats[k])
# end

JS.zbar(xs, cs, p0, 2, mhats, ps, ones(K))

In [ ]:
### Sanity Checks.  
#  # spot check zLOO(alpha) and ZOR(alpha) against other implementation
#  # do a run for large K.
#  #check that oracle value minimized at bayesian value = 2
#  #do a curve by curve spot check

#JS.zbar(xs, cs, 1.515, mhats, ps, ones(K)) 
JS.zLOObar_unsc(xs, cs, p0, 0.44717265850424653, mhats)

In [ ]:
JS.mse_estimates(mhats, collect(1:d), p0, alpha_grid)

In [ ]:
println( alpha_grid[indmin(outOR)], "\t",  alpha_grid[indmin(outLOO)] )
println(minimum(outOR), "\t", outOR[indmin(outLOO)]/minimum(outOR), "\t", outOR[21]/minimum(outOR))


### Implementing Cross-Validation

In [15]:
using Distributions, Plots, Random
include("../src/JS_SAA_main.jl")
pyplot()

Plots.PyPlotBackend()

In [16]:
K = 20
d = 11
N = 7
Random.seed!(8675309)

#gen an "interesting" distribution of ps still centered at 1/d
p0 = ones(d)/d
ps = rand(Dirichlet(ones(d)), K)
Nhats = rand(Poisson(N), K)
mhats = JS.sim_path(ps, Nhats);

In [ ]:
@time cv_data = split_cv(mhats, 5)

[minimum(sum(cv_data[fold], dims=1)) for fold = 1:5]


In [ ]:
fold = 5
train_data = sum(cv_data[setdiff(1:5, fold)])
test_Nhats = sum(cv_data[fold], dims=1)
Nhat_filt = vec(test_Nhats .> 0)
test_phat = zero(cv_data[fold])
test_phat[:, Nhat_filt] = cv_data[fold][:, Nhat_filt] ./ transpose(test_Nhats[Nhat_filt])

findall(sum(test_phat, dims=1) .< 1)
test_Nhats[13:15]


In [ ]:
ones(10)